In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import os

import scanpy as sc
import anndata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.metrics import adjusted_rand_score as ARI
from sklearn.metrics import adjusted_mutual_info_score as AMI

from scipy import sparse
from matplotlib import rcParams
rcParams['pdf.fonttype'] = 42 # enables correct plotting of text
rcParams['figure.figsize'] = (12,12)
import seaborn as sns
from scipy import sparse

sc.settings.verbosity =0

In [2]:
mod ='atac'

In [3]:
adata = sc.read('/home/macera/beegfs/backup_laptop/Documents/CZI/MODALITY_INTEGRATION/objects/atac.h5ad', compression='gzip')

/home/macera/.conda/envs/scanpy_basic/lib/python3.9/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [5]:
adata
samples_to_filter = ['lib_23','lib_38']

adata = adata[~adata.obs['sample'].isin(samples_to_filter)].copy()

/home/macera/.conda/envs/scanpy_basic/lib/python3.9/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [6]:
adata

AnnData object with n_obs × n_vars = 62055 × 152291
    obs: 'sample', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'nucleosome_signal', 'tss_score', 'Deepscore_Annotation', 'Imputed_Deepscore_external', 'Imputed_Deepscore_external_score', 'lowQ_cells'
    var: 'gene_ids', 'feature_types', 'genome', 'interval', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'dispersions', 'dispersions_norm'
    uns: 'AMULET_doublets_colors', 'Deepscore_HVG_GA_colors', 'Deepscore_HVG_RNA_colors', 'Deepscore_markers_GA_colors', 'Deepscore_markers_RNA_colors', 'additional_doublets_colors', 'bySample_leiden_Celltype_colors', 'celltype_colors', 'hvg', 'leiden', 'leiden_colors', 'lsi', 'merged_doublets_colors', 'neighbors', 'rank_genes_groups', 'sample_colors', 'umap'
    obsm: 'MultiVI_latent', 'X_hvp_lsi_harmoniz

In [13]:
obs = pd.read_csv('/home/macera/beegfs/backup_laptop/Documents/CZI//MODALITY_INTEGRATION/Multi_obs.csv', index_col=0)

/tmp/ipykernel_19907/3869542942.py:1: DtypeWarning: Columns (4,5,6,11,13) have mixed types. Specify dtype option on import or set low_memory=False.
  obs = pd.read_csv('/home/macera/beegfs/backup_laptop/Documents/CZI//MODALITY_INTEGRATION/Multi_obs.csv', index_col=0)


In [10]:
obs

,sample,batch,modality,Annotation_bySample,bySample_leiden_Celltype,kidney_area,_indices,_scvi_batch,_scvi_labels,leiden,...,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,formated_index,xtra,Imputed_Deepscore_external,Imputed_Deepscore_external_score
AAACAGCCAACACCTA-3-snRNA_accessibility,lib_15,snRNA,accessibility,PT,PT,Cortex,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,AAACAGCCAACACCTA-3-snRNA_accessibility-0-0,0,PT,0.999994
AAACAGCCAAGGCCAA-13-snRNA_accessibility,lib_57,snRNA,accessibility,PT,PT,Cortex,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,AAACAGCCAAGGCCAA-13-snRNA_accessibility-0-0,0,PT,0.999934
AAACAGCCAATAACGA-13-snRNA_accessibility,lib_57,snRNA,accessibility,PT,PT,Cortex,2,0,0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,AAACAGCCAATAACGA-13-snRNA_accessibility-0-0,0,PT,0.996400
AAACAGCCACAGCCAT-9-snRNA_accessibility,lib_51,snRNA,accessibility,FIB,FIB,Cortex,3,0,0,12,...,NaN,NaN,NaN,NaN,NaN,NaN,AAACAGCCACAGCCAT-9-snRNA_accessibility-0-0,0,FIB,1.000000
AAACAGCCACCAAAGG-10-snRNA_accessibility,lib_54,snRNA,accessibility,PT_VCAM1,PT_VCAM1,Cortex,4,0,0,7,...,NaN,NaN,NaN,NaN,NaN,NaN,AAACAGCCACCAAAGG-10-snRNA_accessibility-0-0,0,PT_VCAM1,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGGTTTCAAAGTAG-1-3-scRNA5p_expression,lib_09,scRNA5p,expression,NaN,NaN,NaN,140424,1,0,4,...,698.0,1460.0,171.0,11.712329,1.0,0.068493,TTTGGTTTCAAAGTAG-1-3-scRNA5p_expression,1,PT,1.000000
TTTGTCAAGTCGTTTG-1-3-scRNA5p_expression,lib_09,scRNA5p,expression,NaN,NaN,NaN,140425,1,0,6,...,2302.0,6300.0,511.0,8.111111,0.0,0.000000,TTTGTCAAGTCGTTTG-1-3-scRNA5p_expression,1,LEUK,1.000000
TTTGTCACACACCGAC-1-3-scRNA5p_expression,lib_09,scRNA5p,expression,NaN,NaN,NaN,140426,1,0,8,...,239.0,691.0,436.0,63.096960,0.0,0.000000,TTTGTCACACACCGAC-1-3-scRNA5p_expression,1,ENDO,0.999997
TTTGTCACACATGACT-1-3-scRNA5p_expression,lib_09,scRNA5p,expression,NaN,NaN,NaN,140427,1,0,27,...,2759.0,8151.0,1390.0,17.053123,4.0,0.049074,TTTGTCACACATGACT-1-3-scRNA5p_expression,1,LEUK,1.000000


In [11]:
obs['modality'].unique()

array(['accessibility', 'expression', 'paired'], dtype=object)

In [ ]:
obs = obs[(obs['batch'].isin(['snRNA']) & obs['modality'].isin(['accessibility','paired']))]
obs.index = [i.split('_')[0] for i in obs.index]
obs.index = [i.replace('-snRNA','') for i in obs.index]
obs

,sample,batch,modality,Annotation_bySample,bySample_leiden_Celltype,kidney_area,_indices,_scvi_batch,_scvi_labels,leiden,...,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,formated_index,xtra,Imputed_Deepscore_external,Imputed_Deepscore_external_score
AAACAGCCAACACCTA-3-snRNA,lib_15,snRNA,accessibility,PT,PT,Cortex,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,AAACAGCCAACACCTA-3-snRNA_accessibility-0-0,0,PT,0.999994
AAACAGCCAAGGCCAA-13-snRNA,lib_57,snRNA,accessibility,PT,PT,Cortex,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,AAACAGCCAAGGCCAA-13-snRNA_accessibility-0-0,0,PT,0.999934
AAACAGCCAATAACGA-13-snRNA,lib_57,snRNA,accessibility,PT,PT,Cortex,2,0,0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,AAACAGCCAATAACGA-13-snRNA_accessibility-0-0,0,PT,0.996400
AAACAGCCACAGCCAT-9-snRNA,lib_51,snRNA,accessibility,FIB,FIB,Cortex,3,0,0,12,...,NaN,NaN,NaN,NaN,NaN,NaN,AAACAGCCACAGCCAT-9-snRNA_accessibility-0-0,0,FIB,1.000000
AAACAGCCACCAAAGG-10-snRNA,lib_54,snRNA,accessibility,PT_VCAM1,PT_VCAM1,Cortex,4,0,0,7,...,NaN,NaN,NaN,NaN,NaN,NaN,AAACAGCCACCAAAGG-10-snRNA_accessibility-0-0,0,PT_VCAM1,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGGTTGAGCCG-3-snRNA,lib_15,snRNA,paired,NaN,NaN,NaN,131114,2,0,11,...,1049.0,1649.0,68.0,4.123711,13.0,0.788357,TTTGTTGGTTGAGCCG-3-snRNA_paired,1,DCT1,0.999425
TTTGTTGGTTGGATAT-9-snRNA,lib_51,snRNA,paired,NaN,NaN,NaN,131115,2,0,14,...,1239.0,1904.0,6.0,0.315126,7.0,0.367647,TTTGTTGGTTGGATAT-9-snRNA_paired,1,CNT,1.000000
TTTGTTGGTTTACTTG-3-snRNA,lib_15,snRNA,paired,NaN,NaN,NaN,131117,2,0,14,...,2201.0,6276.0,1836.0,29.254303,68.0,1.083493,TTTGTTGGTTTACTTG-3-snRNA_paired,1,DCT2,0.997922
TTTGTTGGTTTCCGGC-3-snRNA,lib_15,snRNA,paired,NaN,NaN,NaN,131118,2,0,7,...,2562.0,5723.0,633.0,11.060633,62.0,1.083348,TTTGTTGGTTTCCGGC-3-snRNA_paired,1,PT_VCAM1,1.000000


In [17]:
adata.obs.index

Index(['AAACAGCCAACACCTA-3', 'AAACAGCCAAGGCCAA-13', 'AAACAGCCAATAACGA-13',
       'AAACAGCCACAGCCAT-9', 'AAACAGCCACCAAAGG-10', 'AAACAGCCACCAAAGG-9',
       'AAACAGCCACTGGCCA-13', 'AAACAGCCACTTAGGC-10', 'AAACAGCCAGCACGTT-12',
       'AAACAGCCAGGTTACC-13',
       ...
       'TTTGTTGGTGACCTGG-6', 'TTTGTTGGTGCTGTAA-9', 'TTTGTTGGTGTCACGG-3',
       'TTTGTTGGTTAGGATT-9', 'TTTGTTGGTTCCGGCT-9', 'TTTGTTGGTTGAGCCG-3',
       'TTTGTTGGTTGGATAT-9', 'TTTGTTGGTTTACTTG-3', 'TTTGTTGGTTTCCGGC-3',
       'TTTGTTGGTTTCCTCC-11'],
      dtype='object', name='index', length=62055)

In [20]:
adata.obs['In_final_obj'] = [i in obs.index for i in adata.obs.index]

In [19]:
adata

AnnData object with n_obs × n_vars = 62055 × 152291
    obs: 'sample', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'nucleosome_signal', 'tss_score', 'Deepscore_Annotation', 'Imputed_Deepscore_external', 'Imputed_Deepscore_external_score', 'lowQ_cells', 'In_final_obj'
    var: 'gene_ids', 'feature_types', 'genome', 'interval', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'dispersions', 'dispersions_norm'
    uns: 'AMULET_doublets_colors', 'Deepscore_HVG_GA_colors', 'Deepscore_HVG_RNA_colors', 'Deepscore_markers_GA_colors', 'Deepscore_markers_RNA_colors', 'additional_doublets_colors', 'bySample_leiden_Celltype_colors', 'celltype_colors', 'hvg', 'leiden', 'leiden_colors', 'lsi', 'merged_doublets_colors', 'neighbors', 'rank_genes_groups', 'sample_colors', 'umap'
    obsm: 'MultiVI_latent', 'X_

In [18]:
adata = adata[adata.obs['In_final_obj'].astype(bool)].copy()

In [21]:
adata.obs['sample'].value_counts()

sample
lib_57    11450
lib_15     7864
lib_51     6867
lib_36     6448
lib_55     5338
lib_29     5273
lib_34     4598
lib_56     4339
lib_09     4196
lib_54     3265
lib_10     2417
Name: count, dtype: int64

In [ ]:
adata.write('objects/atac.h5ad', compression='gzip')

In [ ]:
adata.obs.to_csv('csv/atac_cells.csv')

### After computing the 10xMultiome RNA object

In [ ]:
rna = sc.read('objects/snRNA_raw.h5ad')
rna.index = [i.replace('-snRNA','') for i in obs.index]
atac_multi = adata[adata.obs.isin(rna.index)].copy()
atac_multi.write('objects/atac_multi.h5ad', compression ='gzip')
adata.obs.to_csv('csv/multiome_cells.csv')